In [ ]:
#donwload pretrained models
!gdown -O models.zip 1dohuMdbD_NWsFb-RFw8GJwPgD_vVmMqQ
!unzip -qqnd workspace models.zip

# Prepare datasets

In [ ]:
# generate resized datasets from page 5
!python gen_p5_datasets.py --stage test --raw_img_dir workspace/data/test/whole_slides --meta_csv_path workspace/data/test/test_metadata.csv


##### EXTRACTING PAGE 5 DATASETS #####
##### DONE EXTRACTING PAGE 5 DATASETS TOTAL TIME: 0:00:02.139800 #####


In [ ]:
# extract tiles
!python gen_masks.py --stage test --raw_img_dir workspace/data/test/whole_slides --meta_csv_path workspace/data/test/test_metadata.csv
!python gen_tiles_56_p3_224.py --stage test --raw_img_dir workspace/data/test/whole_slides --meta_csv_path workspace/data/test/test_metadata.csv
!python gen_tiles_40_p3_320.py --stage test --raw_img_dir workspace/data/test/whole_slides --meta_csv_path workspace/data/test/test_metadata.csv


##### MASKING SLIDES #####
Applying filters to images (multiprocess)

Number of processes: 2 Number of training images: 16
Time to apply filters to all images (multiprocess): 0:00:30.031535

##### TIME MASKING SLIDES: 0:00:30.051287 #####
##### GENERATING TILE META 56 tile sizes: {0: 1792, 1: 896, 2: 448, 3: 224, 4: 112} #####
Number of processes: 2 Number of training images: 16
##### GENERATING TILES 56_3_224 #####
##### DONE GENERATING TILES 56_3_224 TOTAL TIME: 0:00:32.646916 #####
##### GENERATING TILE META 40 tile sizes: {0: 1280, 1: 640, 2: 320, 3: 160, 4: 80} #####
Number of processes: 2 Number of training images: 16
##### GENERATING TILES 40_2_320 #####
##### DONE GENERATING TILES 40_2_320 TOTAL TIME: 0:00:41.398395 #####


In [ ]:
# generate embeddings
!python gen_embeddings.py --expr expr_56_3_224 --stage test --meta_csv_path workspace/data/test/test_metadata.csv
!python gen_embeddings.py --expr expr_40_2_320 --stage test --meta_csv_path workspace/data/test/test_metadata.csv

tile directory: ./workspace/data/test/tiles/56/3_224
embedder_model_path: workspace/models/resnet18-f37072fd.pth
##### GENERATING EMBEDDINGS expr_56_3_224, device: cuda n_tiles: 3559  #####
2 427 746 746
##### DONE EMBEDDING expr_56_3_224 TOTAL TIME: 0:00:33.484483 #####
tile directory: ./workspace/data/test/tiles/40/2_320
embedder_model_path: workspace/models/resnet18-f37072fd.pth
##### GENERATING EMBEDDINGS expr_40_2_320, device: cuda n_tiles: 3364  #####
21 342 512 511
##### DONE EMBEDDING expr_40_2_320 TOTAL TIME: 0:00:37.111578 #####


# Inference

In [ ]:
#generate test swin model predictions
for EXPR in ['swin384','swin256']:
  !python gen_preds_swin_test.py --expr {EXPR} --meta_csv_path workspace/data/test/test_metadata.csv 


GENERATING p5 swin384 swin_large_patch4_window12_384 PREDICTIONS
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GENERATING p5 swin256 swinv2_base_window12to16_192to256_22kft1k PREDICTIONS
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
#generate test predictions for factors (breslow, ulceration)
for EXPR,SEED in [('expr_56_3_224',41),('expr_40_2_320',320)]:
  for VAR in ['breslow','ulceration']:
    !python gen_factors_test.py --expr {EXPR} --var {VAR} --seed {SEED} --meta_csv_path workspace/data/test/test_metadata.csv 

VMelTilesDataset torch.Size([251, 512]) torch.Size([251, 2]) 16
GENERATING expr_56_3_224 breslow predictions
VMelTilesDataset torch.Size([251, 512]) torch.Size([251, 2]) 16
GENERATING expr_56_3_224 ulceration predictions
VMelTilesDataset torch.Size([332, 512]) torch.Size([332, 2]) 16
GENERATING expr_40_2_320 breslow predictions
VMelTilesDataset torch.Size([332, 512]) torch.Size([332, 2]) 16
GENERATING expr_40_2_320 ulceration predictions


In [ ]:
#generate test mlp predictions 
for EXPR in ['expr_56_3_224','expr_40_2_320']:
  !python gen_preds_mil_test.py --expr {EXPR} --meta_csv_path workspace/data/test/test_metadata.csv 

factors (16, 3) data (16, 4)
factors (16, 3) data (16, 4)


In [ ]:
!python create_submission.py --submission_format_csv_path workspace/data/test/submission_format.csv

writting submission to submission.csv
